In [7]:
from papermill import execute_notebook, PapermillExecutionError
import pandas as pd
import pathlib
import numpy as np

## Setup

In [11]:
dmr_prefix = 'DGmCH'
dmr_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/'
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/axis-mch_rms_results_collapsed.tsv'

delta_cutoff=0.3
dms_cutoff = 1

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'

motif_file_path='/home/hanliu/ref/MEME/selected_motifs_sets/JASPAR2020_CORE_vertebrates_non-redundant_pfms.has_mouse_id.meme',
genome_fasta_path='/home/hanliu/ref/mouse/genome/fasta/raw/mm10.fa',
chrom_size_path='/home/hanliu/ref/mouse/genome/mm10.main.chrom.sizes'
    

In [12]:
dmr_dir = pathlib.Path(dmr_dir)
recipe_dir = dmr_dir / 'Recipe'
recipe_dir.mkdir(exist_ok=True)

## Step1 Filter DMR

In [ ]:
params = dict(
dmr_path = dmr_path,
dmr_prefix = dmr_prefix,
black_list_path = black_list_path,
delta_cutoff=delta_cutoff,
dms_cutoff = dms_cutoff
)

input_path = '1.FilterDMRRecipe_simple_min_max.ipynb'
output_path = recipe_dir / '1.FilterDMRRecipe_simple_min_max.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=params,
    engine_name=None,
    prepare_only=False,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(recipe_dir))
pass

## Step2 MotifScan

In [ ]:
params = dict(bed_path=str(dmr_dir / 'TotalDMR.nofilter.bed'),
              motif_file_path=motif_file_path,
              genome_fasta_path=genome_fasta_path,
              chrom_size_path=chrom_size_path)

input_path = '2.MotifScan.ipynb'
output_path = recipe_dir / '2.MotifScan.ipynb'
execute_notebook(str(input_path),
                 str(output_path),
                 parameters=params,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=str(recipe_dir))
pass

## Step3 Genome Feature Annotation

In [ ]:
params = dict(
    dmr_bed = str(dmr_dir / 'TotalDMR.nofilter.bed')
)
input_path = '3.DMRAnnotation.ipynb'
output_path = recipe_dir / '3.DMRAnnotation.ipynb'
execute_notebook(str(input_path),
                 str(output_path),
                 parameters=params,
                 engine_name=None,
                 prepare_only=False,
                 kernel_name=None,
                 progress_bar=True,
                 log_output=False,
                 start_timeout=60,
                 report_mode=False,
                 cwd=str(recipe_dir))
pass

## Step 4 motif enrichment

In [13]:
groups = ['high', 'low', 'midhigh', 'midlow']
dmr_type = 'DMR'
for group in groups:
    params = dict(
        mask_quantile_to99=0.9,
        or_cutoff=1.5,
        neg_lgp_cutoff=10,
        group=group,
        motif_hits_path=
        f'/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/{dmr_type}/MotifScan.h5ad',
        dmr_hits_path=
        f'/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/{dmr_type}/HypoDMR/{group}.DMS1.bed',
        output_dir=
        f'/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/{dmr_type}/MotifEnrichment'
    )
    
    input_path = '4.MotifEnrichment.ipynb'
    output_path = recipe_dir / f'4.MotifEnrichment.{group}.ipynb'
    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=params,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(recipe_dir))
pass